In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from matplotlib.colors import ListedColormap
import itertools
import random
from sklearn.metrics import adjusted_rand_score
import time
from tqdm import tqdm
import random

In [ ]:
!git clone https://github.com/mikey-joyce/Glaucoma.git
!ls

fatal: destination path 'Glaucoma' already exists and is not an empty directory.
drive  Glaucoma  sample_data


In [ ]:
from Glaucoma.algorithms.unsupervised.BorderClustering import BorderClustering
from Glaucoma.algorithms.unsupervised.FCM import FCM
from Glaucoma.algorithms.unsupervised.PCM import PCM

from Glaucoma.algorithms.data.Retina import RetinaModel

from sklearn.mixture import GaussianMixture

In [ ]:
rm_features = [
    'IOP', 'MAP', 'SBP', 'DBP', 'HR', 'Qmean', 'P1', 'P2', 'P4', 'P5', 'R4', 'R5'
]

octa_features = [
    'IOP', 'MAP', 'SBP', 'DBP', 'HR',
    'Whole I Small Vessels (SV)',
    'Inside Disc  (SV)',
    'Peripapillary  ALL',
    'Superior Hemi ALL',
    'Inferior Hemi  ALL',
    'I half clock',
    'VesselDens SN',
    'VesselDens TI'
]

data_dir = '/content/drive/My Drive/MSSM/'
plot_dir = data_dir + 'exhaustive/results/'

data = pd.read_csv(data_dir + 'sinai.csv')
oct = pd.read_csv(data_dir + 'sinai_octa.csv')
data = pd.merge(data, oct, on='Patient')
# data = data[data['Glaucoma'] == True]

data['IOP']

,IOP
0,13
1,12
2,10
3,14
4,14
...,...
275,18
276,14
277,16
278,14


In [ ]:
train_X = data[rm_features]
train_X = train_X.dropna()

data = data.drop(index=259)
train_X = train_X.drop(index=259) # for some reason this patient seems to be bad

print(len(train_X))

data = data.drop(index=279)
train_X = train_X.drop(index=279)

data2augment = train_X.copy()

s = MinMaxScaler()
train_X = s.fit_transform(train_X)
train_X

279


array([[2.90322581e-01, 2.65822785e-01, 2.36842105e-01, ...,
        2.68985899e-01, 5.27987312e-01, 3.98283736e-04],
       [2.58064516e-01, 4.30379747e-01, 4.56140351e-01, ...,
        4.85821117e-01, 3.64363506e-01, 3.11153536e-04],
       [1.93548387e-01, 1.26582278e-01, 8.77192982e-02, ...,
        1.27840406e-01, 4.23559601e-01, 3.14346352e-04],
       ...,
       [3.22580645e-01, 3.54430380e-01, 2.89473684e-01, ...,
        3.32743714e-01, 5.56684776e-01, 4.22878746e-04],
       [3.87096774e-01, 4.30379747e-01, 4.21052632e-01, ...,
        4.47064495e-01, 6.21933576e-01, 4.75506627e-04],
       [3.22580645e-01, 3.92405063e-01, 5.17543860e-01, ...,
        5.14425083e-01, 4.69150504e-01, 3.81965978e-04]])

In [ ]:
def compute_map(sbp, dbp):
    """Compute Mean Arterial Pressure (MAP)."""
    return dbp + (1/3) * (sbp - dbp)

start = time.time()

resulting_df = pd.DataFrame(columns=rm_features)

means, covars, weights = [], [], []
for i, row in tqdm(data2augment.iterrows(), total=data2augment.shape[0]):
  curr_iop = row['IOP']
  curr_sbp = row['SBP']
  curr_dbp = row['DBP']
  curr_hr = row['HR']

  length = 10
  iop_range, pressure_range, hr_range = 3, 10, 10
  iop_samples = [float(curr_iop + random.randint(-iop_range, iop_range)) for _ in range(length)]
  sbp_samples = [float(curr_sbp + random.randint(-pressure_range, pressure_range)) for _ in range(length)]
  dbp_samples = [float(curr_dbp + random.randint(-pressure_range, pressure_range)) for _ in range(length)]
  hr_samples = [float(curr_hr + random.randint(-hr_range, hr_range)) for _ in range(length)]

  map_samples = []
  for j in range(len(sbp_samples)):
    map_samples.append(compute_map(sbp_samples[j], dbp_samples[j]))

  temp = pd.DataFrame({
      'IOP': iop_samples,
      'MAP': map_samples,
      'SBP': sbp_samples,
      'DBP': dbp_samples,
      'HR': hr_samples
  })

  rm = RetinaModel(
      icp='/content/Glaucoma/config/Initial_wIOP.csv',
      fn=temp,
      ret_fn=0
  )
  res = rm.pipeline()

  res = res.drop(columns=['R1'])
  row_df = pd.DataFrame([row], columns=rm_features)
  temp_df = pd.concat([row_df, res], ignore_index=True)

  resulting_df = pd.concat([resulting_df, temp_df], ignore_index=True)

  temp_df = s.transform(temp_df)

  gmm = GaussianMixture(n_components=1, random_state=42)
  gmm.fit(temp_df)

  # print("Means of the Gaussian components:")
  # print(gmm.means_)
  # print("\nCovariances of the Gaussian components:")
  # print(gmm.covariances_)
  # print("\nWeights of the Gaussian components:")
  # print(gmm.weights_)

  means.append(gmm.means_)
  covars.append(gmm.covariances_)
  weights.append(gmm.weights_)

end = time.time()
elapsed = end - start
print(f"Elapsed time: {elapsed}s")

means = np.array(means)
covars = np.array(covars)
weights = np.array(weights)

resulting_df

  0%|          | 1/278 [00:15<1:13:21, 15.89s/it]

Done!


  1%|          | 2/278 [00:33<1:17:48, 16.91s/it]

Done!


  1%|          | 3/278 [00:50<1:18:07, 17.05s/it]

...

 99%|█████████▉| 276/278 [1:21:49<00:35, 17.58s/it]

Done!
Done!


100%|██████████| 278/278 [1:22:23<00:00, 17.78s/it]

Done!
Elapsed time: 4943.723776578903s


,IOP,MAP,SBP,DBP,HR,Qmean,P1,P2,P4,P5,R4,R5
0,13.0,88.000000,113.0,75.0,96.0,0.000646,41.099485,33.755000,21.110527,18.580625,4473.475249,2232.200139
1,15.0,85.333333,110.0,73.0,96.0,0.000618,40.289402,33.254165,20.980497,18.384330,5034.120124,2235.617519
2,16.0,93.666667,115.0,83.0,102.0,0.000668,42.479982,34.881943,21.582662,18.736160,5146.846027,2236.639078
3,16.0,92.333333,109.0,84.0,92.0,0.000632,41.032160,33.836099,21.212594,18.483294,5255.543658,2237.039331
4,15.0,91.666667,117.0,79.0,92.0,0.000675,42.580684,34.906428,21.564011,18.785655,4873.988734,2234.988538
...,...,...,...,...,...,...,...,...,...,...,...,...
3053,17.0,93.000000,141.0,69.0,54.0,0.000802,47.961496,38.863354,23.001363,19.683368,5077.271294,2236.740452
3054,14.0,94.666667,138.0,73.0,50.0,0.000799,47.316428,38.250114,22.702090,19.665909,4368.003200,2232.085065
3055,14.0,100.666667,148.0,77.0,45.0,0.000872,50.174134,40.294386,23.413195,20.179660,4215.414043,2231.302598
3056,15.0,98.000000,136.0,79.0,41.0,0.000796,47.380715,38.340961,22.760117,19.646864,4602.410776,2233.662361


In [ ]:
# np.save(plot_dir+'spaces.npy', spaces)
np.save(plot_dir+'means.npy', means)
np.save(plot_dir+'covars.npy', covars)
np.save(plot_dir+'weights.npy', weights)

resulting_df.to_csv(plot_dir+'augmented.csv', index=False)